In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
import numpy as np

'''
Preparing data
'''
import pandas as pd
import time
import matplotlib.pyplot as plt
import math

'''
读入一支股票指定年份的ohlcv数据
输入:baseDir,stockCode为字符, startYear,yearNum为整数，
输出:dataframe
'''
def readWSDFile(baseDir, stockCode, startYear, yearNum=1, usecols=None, 
                names=['date','pre_close','open','high','low','close','change','chg_range',
                                               'volume','amount','turn']):
    # 解析日期
    filename = baseDir+stockCode+'/'+stockCode+'.csv'
    print (filename, "===============")
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()
    df = pd.read_csv(filename, index_col=0, sep=',', header=None,usecols=usecols,
                            skiprows=1, names=names,
                           parse_dates=True, date_parser=dateparse)
    return df['2005-01-04':'2015-12-31']

'''
读入一支股票指定年份的技术指标
输入:baseDir,stockCode为字符, startYear,yearNum为整数，
输出:dataframe
'''
def readWSDIndexFile(baseDir, stockCode, startYear, yearNum=1):
    # 解析日期
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()

    df = 0
    for i in range(yearNum):
        tempDF = pd.read_csv(baseDir+'I'+stockCode+'/wsd_'+stockCode+'_'+str(startYear+i)+'.csv', index_col=0, sep=',', parse_dates=True, date_parser=dateparse
                             # , usecols=usecols
                             )
        if i==0: df = tempDF
        else: df = df.append(tempDF)
    return df

# 整理好多因子输入，以dataframe返回数据+标签
from sklearn import preprocessing
def data_prepare(retrace = 0.618):
    # prepare data
    baseDir = '../data/'
    stockCodes = ['000300.SH']
    i = 0
    startYear = 2005
    number =11
    usecols = None#[0,5,6]
    names = ['date','close','change']
    df = readWSDFile(baseDir, stockCodes[i], startYear, number, usecols)
    #dfi = readWSDIndexFile(baseDir, stockCodes[i], startYear, number)
    allDF = df#pd.concat([df, dfi], axis=1)
    sample_num = np.shape(df)[0]
    labelDF = pd.Series(np.zeros(sample_num))
    print ("Factors Shape:", np.shape(df), np.shape(allDF))
    
    # 求出 trend
    price = df['close']
    start = 0
    while price[start] > price[start+1]:
        labelDF[start] = 1 #flat
        start +=1
    print("----- start: ",start)
    #find peak, find trough, calculate retracement and label trend accordingly
    i = start
    while i < sample_num - 1:
        cursor = i
        while cursor < sample_num - 1 and price[cursor] <= price[cursor+1]:
            cursor += 1
        peak = cursor
        while cursor < sample_num - 1 and price[cursor] >= price[cursor+1]:
            cursor += 1
        trough = cursor
        retracement = (price[peak] - price[trough]) / (price[peak] - price[i])
        mark = 1 # flat
        if retracement < retrace:
            mark = 2 # UP
        elif retracement > 1 + retrace:
            mark = 0 # DOWN
        for k in range(i, cursor+1):
            labelDF[k] = mark
        i = cursor

    print("---- Trend Distribution Check --------")
    print(labelDF.value_counts().sort_index())
    
    # make a deep copy of Price Difference before normalizing
    priceDF = allDF['change'].copy(deep=True)
    # scikit-learn normalize or: keras.utils.normalize(x)
    scaler = preprocessing.MinMaxScaler()
    input_data = scaler.fit_transform(allDF)
    print ("input data shape: ", np.shape(input_data)) #  days *  factors
    print ("input label shape: ", np.shape(labelDF))
   
    return input_data, labelDF, priceDF # train/test data, labels and prices for yield calucluation

###### Hyper paramters #########
time_steps = 8
n_neurons = 300
num_classes = 3
batch_size = 20 # specify batch size explicitly; no shuffle but successive sequence
n_epoch = 75
# get training data
train_ratio = 0.9
#_, dataset, _ = data_prepare()
dataset,labels, _ = data_prepare()
segment_num = (len(dataset) - time_steps - 1) // batch_size # rollingly use data
train_size = int(segment_num * train_ratio)
test_size = segment_num - train_size
data_dim = 1#np.shape(dataset)[1] #input

#divide training/validation dataset
#train_x = dataset[0 : train_size * batch_size + time_steps]
#test_x = dataset[train_size * batch_size : (train_size + test_size) * batch_size + time_steps]

train_x = labels.iloc[0 : train_size * batch_size + time_steps]
test_x = labels.iloc[train_size * batch_size : (train_size + test_size) * batch_size + time_steps]
#label is just 1 step further after sequence data
train_y = labels.iloc[time_steps : train_size * batch_size + time_steps]
test_y = labels.iloc[train_size * batch_size + time_steps: (train_size + test_size) * batch_size + time_steps]

# construct training data, 1 step forward, keep rolling
train_x = np.array(train_x)
train_sample = len(train_x) - time_steps
b = np.array([[]])
for i in range(train_sample):
    b = np.append(b, train_x[i : time_steps + i])
train_x = b.reshape(train_sample, time_steps, data_dim)
print("training size: ", train_sample)

test_x = np.array(test_x)
test_sample = len(test_x) - time_steps
b = np.array([[]])
for i in range(test_sample):
    b = np.append(b, test_x[i : time_steps + i])
test_x = b.reshape(test_sample, time_steps, data_dim)
print("testing size: ", test_sample)

train_y = np.array(train_y, dtype=np.int32)
test_y = np.array(test_y, dtype=np.int32)


# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(n_neurons, return_sequences=True,stateful=True,
               batch_input_shape=(batch_size, time_steps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(n_neurons, return_sequences=True, stateful=True))  # returns a sequence of vectors of dimension 32
# add 5 more layers
model.add(LSTM(n_neurons, return_sequences=True, stateful=True))

model.add(LSTM(n_neurons, stateful=True))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', #for integer class, not one hot encoding
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=batch_size, 
          epochs=n_epoch,
          shuffle=False,
          validation_data=(test_x, test_y), 
          callbacks=[TensorBoard(log_dir='./logs/pure_clz', write_graph=True, write_images=True)]
         )
print ("Params: ", "time_steps:", time_steps, "  n_neurons:", n_neurons, " n_epoch: ", n_epoch)

../data/000300.SH/000300.SH.csv ===============
Factors Shape: (2671, 10) (2671, 10)
----- start:  0
---- Trend Distribution Check --------
0.0     960
1.0     558
2.0    1153
dtype: int64
input data shape:  (2671, 10)
input label shape:  (2671,)
training size:  2380
testing size:  280
Train on 2380 samples, validate on 280 samples
Epoch 1/75
2380/2380 [==============================] - 13s - loss: 1.1025 - acc: 0.4475 - val_loss: 1.0006 - val_acc: 0.6964
Epoch 2/75
2380/2380 [==============================] - 12s - loss: 0.8288 - acc: 0.6655 - val_loss: 0.6034 - val_acc: 0.7714
Epoch 3/75
2380/2380 [==============================] - 12s - loss: 0.6132 - acc: 0.8080 - val_loss: 0.5399 - val_acc: 0.8679
Epoch 4/75
2380/2380 [==============================] - 12s - loss: 0.5562 - acc: 0.8408 - val_loss: 0.5123 - val_acc: 0.8679
Epoch 5/75
2380/2380 [==============================] - 12s - loss: 0.5371 - acc: 0.8429 - val_loss: 0.5197 - val_acc: 0.8679
Epoch 6/75
2380/2380 [==============

2380/2380 [==============================] - 12s - loss: 0.1061 - acc: 0.9647 - val_loss: 0.8882 - val_acc: 0.7964
Epoch 63/75
2380/2380 [==============================] - 12s - loss: 0.1058 - acc: 0.9601 - val_loss: 1.0234 - val_acc: 0.8000
Epoch 64/75
2380/2380 [==============================] - 12s - loss: 0.0943 - acc: 0.9731 - val_loss: 1.1417 - val_acc: 0.7643
Epoch 65/75
2380/2380 [==============================] - 12s - loss: 0.0812 - acc: 0.9739 - val_loss: 1.0750 - val_acc: 0.7893
Epoch 66/75
2380/2380 [==============================] - 12s - loss: 0.0841 - acc: 0.9689 - val_loss: 1.0868 - val_acc: 0.7464
Epoch 67/75
2380/2380 [==============================] - 12s - loss: 0.0823 - acc: 0.9685 - val_loss: 1.1405 - val_acc: 0.7929
Epoch 68/75
2380/2380 [==============================] - 13s - loss: 0.0697 - acc: 0.9765 - val_loss: 1.1060 - val_acc: 0.7714
Epoch 69/75
2380/2380 [==============================] - 12s - loss: 0.0895 - acc: 0.9689 - val_loss: 1.1626 - val_acc: 0.7

In [ ]:
 - 9s - loss: 0.2482 - acc: 0.9105 - val_loss: 0.6344 - val_acc: 0.8107
    Params:  time_steps: 8   n_neurons: 300  n_epoch:  100
    
    ## +5 layers
 - 30s - loss: 1.0736 - acc: 0.4353 - val_loss: 1.0071 - val_acc: 0.5393
Params:  time_steps: 8   n_neurons: 300  n_epoch:  50              
    
    ## +3 layers
-- 19s - loss: 1.0764 - acc: 0.4290 - val_loss: 1.0129 - val_acc: 0.5393
Params:  time_steps: 8   n_neurons: 300  n_epoch:  75                

In [16]:
from pandas import DataFrame
from pandas import concat
import pandas as pd
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
	# drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def readWSDFile(baseDir, stockCode, startYear, yearNum=1, usecols=[0,1,2], 
                names=['date','pre_close','open']):
    # 解析日期
    filename = baseDir+stockCode+'/'+stockCode+'.csv'
    print (filename, "===============")
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()
    df = pd.read_csv(filename, index_col=0, sep=',', header=None,usecols=usecols,
                            skiprows=1, names=names,
                           parse_dates=True, date_parser=dateparse)
    return df['2015-10-04':'2015-10-31']
a = readWSDFile('../data/','000300.SH',0)
b = series_to_supervised(a)
pd.concat([a,b], axis=1)

../data/000300.SH/000300.SH.csv ===============


,pre_close,open,var1(t-1),var2(t-1),var1(t),var2(t)
date,,,,,,
2015-10-08,3202.95,3324.98,NaN,NaN,NaN,NaN
2015-10-09,3296.48,3302.36,3202.95,3324.98,3296.48,3302.36
2015-10-12,3340.12,3351.14,3296.48,3302.36,3340.12,3351.14
2015-10-13,3447.69,3422.48,3340.12,3351.14,3447.69,3422.48
2015-10-14,3445.04,3431.14,3447.69,3422.48,3445.04,3431.14
2015-10-15,3406.11,3403.39,3445.04,3431.14,3406.11,3403.39
2015-10-16,3486.82,3508.52,3406.11,3403.39,3486.82,3508.52
2015-10-19,3534.07,3548.95,3486.82,3508.52,3534.07,3548.95
2015-10-20,3534.18,3527.88,3534.07,3548.95,3534.18,3527.88
